# 286 - Scoring Probabilities

## Problem Statement

Barbara is a mathematician and a basketball player. She has found that the probability of scoring a point when shooting from a distance $x$ is exactly $(1 - x / q)$, where $q$ is a real constant greater than $50$.

During each practice run, she takes shots from distances $x = 1, x = 2, \dots, x = 50$ and, according to her records, she has precisely a $2\%$ chance to score a total of exactly $20$ points.

Find $q$ and give your answer rounded to $10$ decimal places.

## Solution

The probability distribution of the number of successes in a series of $n$ independent Bernoulli trials with potentially different success probability, $p_1, p_2, \cdots, p_n$, is the Poisson binomial distribution. Here the number of trials is $n = 50$ and the probabilities are given by 

\begin{equation}
    p_x = 1 - \frac{x}{q}.
\end{equation}

We will use this probability distribution to compute the probability of 20 successes among the 50 trials given our $p_x$ values. Unfortunately, for such a large $n$, computing the PMF of the Poisson binomial distribution is not possible. Instead, we will use the discrete Fourier transform where the probability of $k$ successes is given by

\begin{equation}
    P(K = k) = \frac{1}{n + 1} \sum_{\alpha = 0}^n C^{-\alpha k} \prod_{x = 1}^n (1 + (C^\alpha - 1)p_x)
\end{equation}

where

\begin{equation}
    C = \exp \left( \frac{2i\pi}{n + 1} \right)
\end{equation}

and $i = \sqrt{-1}$.

Now, we simply need to implement the function above, subtract 0.02 from the result and numerically solve for the root of the resulting function by changing the value of $q$. For the solver, we will use Brent's method as implemented in `scipy.optimize.brentq`. We arbitrarily choose a range for $q$ between 51 and 100. This can be updated if the root is not found.

In [1]:
import numpy as np
from scipy.optimize import brentq

def poisson_binomial_distribution_dft(n, k, q):
    # Probability of scoring
    p = [1 - (x / q) for x in range(n + 1)]
    # Poisson binomial distribution (discrete Fourier transform)
    C = np.exp((2 * 1j * np.pi) / (n + 1))
    curr_sum = 0
    for i in range(n + 1):
        curr_prod = 1
        for j in range(1, n + 1):
            curr_prod *= (1 + (C**i - 1) * p[j])
        curr_sum += C**(-i * k) * curr_prod
    return (curr_sum / (n + 1)).real

def objective_function(q):
    return poisson_binomial_distribution_dft(50, 20, q) - 0.02

res = brentq(objective_function, 51, 100, xtol=1e-12)
round(res, 10)

52.6494571953